# Darts

An interesting example of a library combining multiple time series models is `darts` https://unit8co.github.io/darts/ . It takes a slightly different approach then AutoTS: instead of automatically combining a set of models from a predefined list, it allows us to setup evaluation of different models in standardized manner.

<img src="darts.png"/>

The list of possible candidate models is quite exhaustive:

* ARIMA
* Auto-ARIMA
* Baseline Models
* Block Recurrent Neural Networks
* Exponential Smoothing
* Fast Fourier Transform
* LightGBM Model
* Linear Regression model
* N-BEATS
* Facebook Prophet
* Random Forest
* Regression ensemble model
* Regression Model
* Recurrent Neural Networks
* Temporal Convolutional Network
* Temporal Fusion Transformer (TFT)
* Theta Method
* Transformer Model
* VARIMA

This package and `Kats` discussed in this section have certain overlap in their focus, so in order to avoid redundancy we will take something of an *a la carte* approach.

In [ ]:
import pandas as pd
from data_sources import *
from darts import TimeSeries
from darts.models import Prophet, ExponentialSmoothing, AutoARIMA, Theta, RegressionEnsembleModel, RegressionModel
from darts.metrics import rmse

In this section we will focus on a very useful feature of darts: backtesting. A frequent approach to validation of time series models is to split the data on time: training set are all the observations up to some time point 𝑡0

and validation is the part afterwards. While simple to set up, this approach runs the risk of overfitting to our chosen validation set. The historical_forecast method built into all the models in darts solves this problem by allowing us to perform a walk-forward validation:
<img src="backtesting.png"/>

For the visually challenged people like me:
   - we use an expanding training window
   - for each run, we train the model on the training set and generate a forecast on test
   - next run has an extended training set and a shifted test

Let's take some of the already familiar models, as well as a few new ones for a ride. First, we borrow a helpful wrapper function from the official documentation:

In [ ]:
def eval_model(model, flow,
               past_covariates=None,  future_covariates=None,
               retrain = False,
               forecast_horizon = 10,
               last_obs = 200):
    # Past and future covariates are optional because they won't always be used in our tests
    # We backtest the model on the last 20% of the flow series,
    # with a horizon of `forecast_horizon` steps:
    backtest = model.historical_forecasts(series=flow,
                                          past_covariates=past_covariates,
                                          future_covariates=future_covariates,
                                          start=0.8,
                                          retrain=retrain,
                                          verbose=True,
                                          forecast_horizon=forecast_horizon)


Format the data into `darts` format:


In [ ]:
stock_names = 'TATAMOTORS'
df = pd.read_csv(data, 'nifty50-stock-market-data', f'{stock_names}.csv', usecols = ['Date', 'VWAP'])
df = df.loc[df.Date > CFG.cutoff]
series = TimeSeries.from_dataframe(df, 'Date', 'VWAP', freq = 'D')
series.plot()
# df.set_index('Date').VWAP.plot()